## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-31-16-55-36 +0000,nypost,JPMorgan CEO Jamie Dimon’s icy relationship wi...,https://nypost.com/2025/07/31/business/jpmorga...
1,2025-07-31-16-55-31 +0000,wapo,"Trump’s tariffs face challenge, while administ...",https://www.washingtonpost.com/business/2025/0...
2,2025-07-31-16-55-21 +0000,nyt,Massachusetts Governor Proposes $400 Million f...,https://www.nytimes.com/2025/07/31/us/politics...
3,2025-07-31-16-54-29 +0000,nyt,Appeals Court Hears Legal Challenge to Trump T...,https://www.nytimes.com/2025/07/31/business/tr...
4,2025-07-31-16-54-14 +0000,nypost,US toy makers are getting slammed by China tar...,https://nypost.com/2025/07/31/business/us-toy-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2381/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
6,trump,75
12,tariffs,24
115,new,20
18,trade,19
79,deal,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
133,2025-07-31-10-33-13 +0000,cbc,"U.S. extends trade deal with Mexico, sets tari...",https://www.cbc.ca/news/world/trump-tariffs-so...,176
249,2025-07-30-22-42-00 +0000,nypost,Trump announces South Korea trade deal with $3...,https://nypost.com/2025/07/30/us-news/trump-an...,161
52,2025-07-31-15-31-00 +0000,wsj,President Trump’s tighter deadline for Russia ...,https://www.wsj.com/politics/policy/trumps-rus...,155
20,2025-07-31-16-29-38 +0000,latimes,New Trump tariffs on Mexico delayed 90 days as...,https://www.latimes.com/world-nation/story/202...,154
239,2025-07-30-23-08-32 +0000,cbc,Trump slams Canada's plan to recognize Palesti...,https://www.cbc.ca/news/politics/us-canada-tra...,145


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
133,176,2025-07-31-10-33-13 +0000,cbc,"U.S. extends trade deal with Mexico, sets tari...",https://www.cbc.ca/news/world/trump-tariffs-so...
315,103,2025-07-30-20-19-00 +0000,wsj,Former Vice President Kamala Harris said Wedne...,https://www.wsj.com/politics/elections/kamala-...
52,64,2025-07-31-15-31-00 +0000,wsj,President Trump’s tighter deadline for Russia ...,https://www.wsj.com/politics/policy/trumps-rus...
82,53,2025-07-31-14-01-19 +0000,nypost,Fed’s preferred inflation gauge ticks up as ta...,https://nypost.com/2025/07/31/business/feds-pr...
253,52,2025-07-30-22-31-15 +0000,nypost,Witkoff could visit Gaza aid sites during Isra...,https://nypost.com/2025/07/30/world-news/witko...
239,51,2025-07-30-23-08-32 +0000,cbc,Trump slams Canada's plan to recognize Palesti...,https://www.cbc.ca/news/politics/us-canada-tra...
361,49,2025-07-30-17-01-29 +0000,wapo,NYC mass killing raises new security concerns ...,https://www.washingtonpost.com/business/2025/0...
308,49,2025-07-30-20-33-00 +0000,wsj,"Ford Takes $800 Million Tariff Hit, Posts Firs...",https://www.wsj.com/business/autos/ford-earnin...
70,45,2025-07-31-14-53-32 +0000,nypost,Newly released video shows horrific moment Arm...,https://nypost.com/2025/07/31/us-news/video-sh...
159,37,2025-07-31-08-34-03 +0000,nypost,"Russian missile and drone attack batters Kyiv,...",https://nypost.com/2025/07/31/world-news/russi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
